In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from sys import exit

Data exploration.

Create a dict of dict and 1 dataframe with the tables joined on subject_id.
Then create a row number partitioned by subject_Id for time series purpose.

In [3]:
'''
Note that this code assumes that there are no data descrepencies between the tables. (unlikely, but for now works.)
'''
def Get_tables(dir:str, currentDir:bool = True, eventsOnly:bool = True, specificFiles:list = []) -> list():
    ''' If CurrentDir set to True, uses current working directory.
    otherwise full path to dir dir must be set.
    If eventsPnly is True, then only Event tables will be selected.
    '''
    if currentDir: os.getcwd() # gets current working dir

    try:
        if eventsOnly:
            return [os.path.join(dir, file) for file in os.listdir(dir) if re.search('EVENTS', file) or re.search('ADMISSIONS', file) and not re.serach('DATETIME')]
        if specificFiles != True and eventsOnly != True: # if specific tables is empty, and eventOnly is false
            out = []
            for filename in os.listdir(dir):
                for i in specificFiles:
                    if re.search(i, filename):
                        out.append(os.path.join(dir, filename))
                        break
            return out
        else:
            return [os.path.join(dir, file) for file in os.listdir(dir)]
    except:
        print('Path "{0}" Not found'.format(dir))

def DataCreation_NestedDict(tables:list):
    ''' Creates a dict of dicts for each table.'''
    table_dict = {}
    if len(tables) == 1:
        return pd.read_csv(tables[0])
    for table in tables:
        file_name = os.path.basename(table)
        table_dict[file_name] = pd.read_csv(table)
    return table_dict

def RemoveDubplicateColumns(tableA: pd.DataFrame, tableB: pd.DataFrame)-> (list, list):
    ''' Helper function''' 
    columns_from_a = tableA.columns
    columns_from_b_not_in_a = [col for col in tableB.columns if col not in tableA.columns]
    return columns_from_a, columns_from_b_not_in_a

def DataCreation_JoinedTables(tables:list, key:str, targetTable = None):
    ''' Creates a dict of dicts for each table.
    the key is the key to join.
    The order of table matters here.
    <targetTable>
    If set, then  this table will be the "left side" of the left join.
    '''
    #if len(tables) == 1 and (targetTable is not None and not targetTable.empty):
    #    return pd.read_csv(tables[0])
    if targetTable is not None and not targetTable.empty:
        init_table = targetTable
    else:
        init_table = pd.read_csv(tables[0]) # set first file as the first init table to join.
    if len(tables) > 1: start_index = 1
    else: start_index = 0
    for table in tables[start_index:]: #don't try and merge the same table to itself. Stupid
        if re.search('ADMISSION', table) != True: #assuming that ADMISSION table is always available.
            new_table = pd.read_csv(table)
            try:
                new_table[key]
            except:
                msg = f'Key: {key} not found for table {os.path.basename(table)}'
                print(msg)
            a, b = RemoveDubplicateColumns(init_table, new_table)
            
            init_table = pd.merge(init_table, new_table, how = 'left', on = key, suffixes=('', '_b'))
            print(len(new_table.keys()))
            init_table = init_table[list(a) + list(b)]
            print(len(init_table.keys()))
    return init_table

Rewrite to handle other file names aswell.

In [4]:
lst = [
         'ADMISSIONS'
         , 'DIAGNOSES_ICD'
        ]
# testing
tables = Get_tables('mimic-iii-clinical-database-demo-1.4', eventsOnly = False, specificFiles = lst)
print(tables)
a = DataCreation_JoinedTables(tables = tables, key = 'subject_id', targetTable= None)
print(len(a.keys()))
a

['mimic-iii-clinical-database-demo-1.4\\ADMISSIONS.csv', 'mimic-iii-clinical-database-demo-1.4\\DIAGNOSES_ICD.csv']
5
21
21


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,seq_num,icd9_code
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,1,99591
1,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,2,99662
2,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,3,5672
3,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,4,40391
4,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,5,42731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,NOT SPECIFIED,SINGLE,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,7,1975
5948,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,NOT SPECIFIED,SINGLE,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,8,45182
5949,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,NOT SPECIFIED,SINGLE,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,9,99592
5950,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,NOT SPECIFIED,SINGLE,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,10,2449


In [6]:
lst2 = [
         'D_ICD_DIAGNOSES'
         #, 'PROCEDUREEVENTS'
]
print(len(a.keys()))
tables = Get_tables('mimic-iii-clinical-database-demo-1.4', eventsOnly = False, specificFiles = lst2)
print(tables)
print(type(a))
b = DataCreation_JoinedTables(tables = tables, key = 'icd9_code', targetTable = a)
print(len(b.keys()))
b

21
['mimic-iii-clinical-database-demo-1.4\\D_ICD_DIAGNOSES.csv']
<class 'pandas.core.frame.DataFrame'>
4
23
23


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,seq_num,icd9_code,short_title,long_title
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,1,99591,Sepsis,Sepsis
1,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,2,99662,React-oth vasc dev/graft,Infection and inflammatory reaction due to oth...
2,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,3,5672,NaN,NaN
3,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,4,40391,Hyp kid NOS w cr kid V,"Hypertensive chronic kidney disease, unspecifi..."
4,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,5,42731,Atrial fibrillation,Atrial fibrillation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,7,1975,Sec malig neo lg bowel,Secondary malignant neoplasm of large intestin...
5948,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,8,45182,Phlbts sprfc vn up extrm,Phlebitis and thrombophlebitis of superficial ...
5949,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,9,99592,Severe sepsis,Severe sepsis
5950,41092,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,WHITE,2170-12-15 02:22:00,2170-12-15 05:25:00,CHOLANGITIS,0,1,10,2449,Hypothyroidism NOS,Unspecified acquired hypothyroidism


In [7]:
b['RN'] = a.sort_values(['subject_id','ethnicity'], ascending=[True,False]) \
             .groupby(['subject_id']) \
             .cumcount() + 1


In [8]:
b[b['subject_id'].isin([10065, 10064])]

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,seq_num,icd9_code,short_title,long_title,RN
294,12314,10064,111761,2127-03-19 14:39:00,2127-03-19 18:09:00,2127-03-19 18:09:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2127-03-19 12:54:00,2127-03-19 15:39:00,SUBDURAL HEMATOMA/S/P FALL,1,1,1,80125,Cl skl base fx-deep coma,Closed fracture of base of skull with subarach...,1
295,12314,10064,111761,2127-03-19 14:39:00,2127-03-19 18:09:00,2127-03-19 18:09:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2127-03-19 12:54:00,2127-03-19 15:39:00,SUBDURAL HEMATOMA/S/P FALL,1,1,2,8602,Traum hemothorax-closed,Traumatic hemothorax without mention of open w...,2
296,12314,10064,111761,2127-03-19 14:39:00,2127-03-19 18:09:00,2127-03-19 18:09:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2127-03-19 12:54:00,2127-03-19 15:39:00,SUBDURAL HEMATOMA/S/P FALL,1,1,3,42731,Atrial fibrillation,Atrial fibrillation,3
297,12314,10064,111761,2127-03-19 14:39:00,2127-03-19 18:09:00,2127-03-19 18:09:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2127-03-19 12:54:00,2127-03-19 15:39:00,SUBDURAL HEMATOMA/S/P FALL,1,1,4,E8348,W/crft fall NEC-pers NEC,Other fall from one level to another in water ...,4
298,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,1,1510,Mal neo stomach cardia,Malignant neoplasm of cardia,1
299,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,2,1978,Sec mal neo GI NEC,Secondary malignant neoplasm of other digestiv...,2
300,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,3,9992,Vasc comp med care NEC,"Other vascular complications of medical care, ...",3
301,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,4,5859,Chronic kidney dis NOS,"Chronic kidney disease, unspecified",4
302,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,5,1962,Mal neo lymph intra-abd,Secondary and unspecified malignant neoplasm o...,5
303,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,NaN,NaN,ESOPHAGEAL CANCER/SDA,0,1,6,7140,Rheumatoid arthritis,Rheumatoid arthritis,6


In [9]:
print(a['row_id'].value_counts())
print(a['ethnicity'].value_counts())


40404    266
40409    266
40405    266
40402    266
40401    266
        ... 
12357      5
12349      5
12285      4
12314      4
12296      3
Name: row_id, Length: 129, dtype: int64
HISPANIC/LATINO - PUERTO RICAN                              3990
WHITE                                                       1555
BLACK/AFRICAN AMERICAN                                       117
UNKNOWN/NOT SPECIFIED                                        115
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE      68
HISPANIC OR LATINO                                            35
OTHER                                                         31
ASIAN                                                         24
UNABLE TO OBTAIN                                              17
Name: ethnicity, dtype: int64


In [ ]:
print(a['icd9_code'].value_counts())

2 table comparison

In [79]:
# lst = [
#         'ADMISSIONS'
#     #    ,'LABEVENTS'
#         ,'NOTEEVENTS'
#        ]
lst = [
          'D_ICD_DIAGNOSES'
]
tables = Get_tables('mimic-iii-clinical-database-demo-1.4', eventsOnly = False, specificFiles = lst)
print(tables)
#a = pd.read_csv(tables[0])
b = pd.read_csv(tables[0])

print(f'{a.keys()}\n{b.keys()}')
aa, bb = RemoveDubplicateColumns(a, b)
print(len(a.keys()), len(b.keys()), len(list(aa)+list(bb)))
print(set(a) & set(b))


['mimic-iii-clinical-database-demo-1.4\\D_ICD_DIAGNOSES.csv']
Index(['row_id', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'seq_num', 'icd9_code'],
      dtype='object')
Index(['row_id', 'icd9_code', 'short_title', 'long_title'], dtype='object')
21 4 23
{'row_id', 'icd9_code'}


In [ ]:
a = DataCreation_NestedDict(tables = tables)
n_col = 0
n_rows = 0
for key, inner_dict in a.items():
    num_rows = len(inner_dict)  # Number of rows
    num_cols = len(inner_dict.keys()) # Number of columns
    print(f"The shape of {key} is {num_rows} rows and {num_cols} columns.")
    n_col += num_cols
    n_rows += num_rows
print(f'Total columns: {n_col}\nTotal rows: {n_rows}') #maybe rows should depend on admission dimensions.


# Data exploration: Data sizes, and errors.

In [ ]:
tables = Get_tables('mimic-iii-clinical-database-demo-1.4')
df = pd.read_csv(tables[0])
print(tables[0])
df

In [ ]:
def __main__():
   tables = Get_tables('mimic-iii-clinical-database-demo-1.4')
  